<a href="https://colab.research.google.com/github/Smruthi3/TSAI_END_NLP/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

We then load the IMDb dataset.

In [2]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 23.1MB/s]


Then create the validation set from our training set.

In [3]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [4]:
len(train_data.examples)

17500

In [5]:
for txt in range(0,len(train_data.examples)):
  vars(train_data.examples[txt]).get('text').reverse()


In [6]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:31, 2.20MB/s]                          
100%|█████████▉| 399561/400000 [00:16<00:00, 23350.24it/s]

In [7]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [8]:
device

device(type='cuda')

In [109]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.ModuleList()

        for i in range(3):
          if(i==0):
            input_dim = embedding_dim
          else:
            input_dim= hidden_dim
          
          self.rnn.append(nn.LSTM(input_dim, 
                            hidden_dim, 
                            num_layers=n_layers, 
                            bidirectional=bidirectional, 
                            dropout=dropout))
        
        self.fc = nn.Linear(hidden_dim * 2 , output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #packed_output=packed_embedded

        for i,layer in enumerate(self.rnn):
          if(i==0):
            packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu().numpy())
          else:
            packed_embedded=packed_output
          
          packed_output, (hidden, cell) = layer(packed_embedded)

        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))

            
        return self.fc(hidden)

In [110]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = False
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [111]:
print(model)

RNN(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (rnn): ModuleList(
    (0): LSTM(100, 256, num_layers=2, dropout=0.2)
    (1): LSTM(256, 256, num_layers=2, dropout=0.2)
    (2): LSTM(256, 256, num_layers=2, dropout=0.2)
  )
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


We'll print out the number of parameters in our model. 

Notice how we have almost twice as many parameters as before!

In [112]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,498,985 trainable parameters


In [113]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [114]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3970,  0.4024,  1.0612,  ..., -0.0136, -0.3363,  0.6442],
        [-0.5197,  1.0395,  0.2092,  ..., -0.8857, -0.2294,  0.1244],
        [ 0.0057, -0.0707, -0.0804,  ..., -0.3292, -0.0130,  0.0716]])

In [115]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3970,  0.4024,  1.0612,  ..., -0.0136, -0.3363,  0.6442],
        [-0.5197,  1.0395,  0.2092,  ..., -0.8857, -0.2294,  0.1244],
        [ 0.0057, -0.0707, -0.0804,  ..., -0.3292, -0.0130,  0.0716]])


## Train the Model

In [116]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [117]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [118]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [119]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [120]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [121]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:

N_EPOCHS = 30

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 55s
	Train Loss: 0.673 | Train Acc: 60.10%
	 Val. Loss: 0.676 |  Val. Acc: 59.48%
Epoch: 02 | Epoch Time: 0m 57s
	Train Loss: 0.672 | Train Acc: 60.37%
	 Val. Loss: 0.676 |  Val. Acc: 59.49%
Epoch: 03 | Epoch Time: 0m 56s
	Train Loss: 0.672 | Train Acc: 60.33%
	 Val. Loss: 0.675 |  Val. Acc: 59.45%
Epoch: 04 | Epoch Time: 0m 57s
	Train Loss: 0.672 | Train Acc: 60.43%
	 Val. Loss: 0.675 |  Val. Acc: 59.55%
Epoch: 05 | Epoch Time: 0m 56s
	Train Loss: 0.672 | Train Acc: 60.43%
	 Val. Loss: 0.676 |  Val. Acc: 59.51%
Epoch: 06 | Epoch Time: 0m 57s
	Train Loss: 0.671 | Train Acc: 60.60%
	 Val. Loss: 0.679 |  Val. Acc: 58.26%
Epoch: 07 | Epoch Time: 0m 56s
	Train Loss: 0.670 | Train Acc: 60.81%
	 Val. Loss: 0.679 |  Val. Acc: 58.58%
Epoch: 08 | Epoch Time: 0m 56s
	Train Loss: 0.676 | Train Acc: 57.67%
	 Val. Loss: 0.689 |  Val. Acc: 55.66%
Epoch: 09 | Epoch Time: 0m 56s
	Train Loss: 0.678 | Train Acc: 58.51%
	 Val. Loss: 0.684 |  Val. Acc: 56.87%
Epoch: 10 | Epoch T

In [123]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.677 | Test Acc: 59.04%


In [124]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [125]:
predict_sentiment(model, "This film is terrible")

0.5011833906173706

In [126]:
predict_sentiment(model, "This film is fantastic")

0.5178212523460388